In [ ]:
# 1. Build the project
dotnet build ClockExtension

In [8]:

using System.IO;

// Create an absolute path since #i doesn't like 
var debugOutputFolder = new DirectoryInfo(@"..\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug").FullName;


var nugetFile = Directory.GetFiles(debugOutputFolder,"*.nupkg")
                      .OrderByDescending(f => File.GetLastWriteTime(f))
                      .FirstOrDefault();

display(debugOutputFolder);

if (File.Exists(Path.Combine(debugOutputFolder, nugetFile)))
{
    "✅ The package is there!".Display();
} 
else
{
    "❌ Something must have gone wrong with the build. The package isn't there.".Display();
}

var nugetSource = $"nuget:{debugOutputFolder}";

c:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug

✅ The package is there!

In [9]:
#r "nuget:Jowsy.DotNet.Interactive.Extensions"

#i @csharp:nugetSource

Restore sources c:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug Installed Packages Jowsy.DotNet.Interactive.Extensions, 1.77.0

In [3]:
#!connect revit --kernel-name revit24 --revit-version 2024

Error: Command cancelled.

In [8]:
#r "nuget:Revit.RevitApi.x64, 2023.0.0"
#r "nuget:Revit.RevitApiUi.x64, 2023.0.0"
using Autodesk.Revit.DB;
using Autodesk.Revit.UI;

Restore sources C:\git\bim-net-interactive\src\Jowsy.DotNet.Interactive.Extensions\bin\Debug Installed Packages Revit.RevitApi.x64, 2023.0.0 Revit.RevitApiUi.x64, 2023.0.0

In [4]:
#!revit24

BuiltInCategory builtInCategory = BuiltInCategory.OST_Walls;
Category category = Category.GetCategory(doc, builtInCategory);
ElementId categoryId = category.Id;

var categoryOverrides = uidoc.ActiveView.GetCategoryOverrides(categoryId);
var patternColor = categoryOverrides.SurfaceForegroundPatternColor;

var color = new { red = patternColor.Red, 
                blue = patternColor.Blue,
                green = patternColor.Green};

color

Requesting value infos...Value info received...Name: uiapp, TypeName: UIApplication: Value:Autodesk.Revit.UI.UIApplication
Name: uidoc, TypeName: UIDocument: Value:Autodesk.Revit.UI.UIDocument
Name: doc, TypeName: Document: Value:Autodesk.Revit.DB.Document


In [5]:
#!set --value @revit24:color --name color

In [9]:
#!revit24

//Because the revit kernel is run out of process embedded in the revit application we need to use the set magic command
// UIDocument uidoc = default;

 Document doc = uidoc.Document;
 
 // Set export options
    IFCExportOptions ifcExportOptions = new IFCExportOptions();
    
    // Optionally, you can customize export options further, e.g., set specific configurations.
    // ifcExportOptions.SpaceBoundaries = IFCExportSpaceBoundaries.Boundaries;
    // ifcExportOptions.SplitWallsAndColumns = true;
    // ...

    // Perform the export
   Transaction transaction = new Transaction(doc, "Export IFC");
   transaction.Start("start");

        // Use the Export method of the Document class

    string fileName = "c:\\playground\\house.ifc";

    doc.Export(System.IO.Path.GetDirectoryName(fileName),System.IO.Path.GetFileName(fileName), ifcExportOptions);

    transaction.Commit();

    fileName

In [6]:
#!connect jupyter --kernel-name pythonkernel --kernel-spec python3

The `#!connect jupyter` feature is in preview. Please report any feedback or issues at https://github.com/dotnet/interactive/issues/new/choose.

Kernel added: #!pythonkernel

In [9]:
#!set --value @revit24:color --name color2

In [ ]:
pip install ifcopenshell

In [ ]:
pip install lark

In [ ]:
pip install numpy

In [11]:
import ifcopenshell



Note that we try to find IfcSolidModel representations. A wall for example might contain a PolyLine and a IfcSweptSolid. IfcSweptSolid is a subtype of IfcSolidModel. To understand the relationship between IfcSweptSolid and IfcSolidModel, it's important to note that IFC uses an inheritance model where subtypes inherit properties and relationships from their supertypes.


https://forums.buildingsmart.org/t/need-help-with-revit-2x3-ifc-export-trying-to-figure-out-how-the-colors-are-assigned/4461/4

In [14]:

# Load an IFC file
ifc_file = ifcopenshell.open('c:\playground\house.ifc')


for element in ifc_file.by_type('IfcProduct'):
    # Check the category of the element, for example, by using the element's 'IfcType'
    element_type = element.is_a()

    if (element_type == "IfcWallStandardCase"):

        display(element_type)
        # Iterate over the wall's representations to apply the style
        for representation in element.Representation.Representations:
            
            for representation_item in representation.Items:
            # Check if the representation item is a subtype of IfcSolidModel

                if representation_item.is_a('IfcSolidModel'):
                    
                    color = ifc_file.createIfcColourRgb(None, color2['red'], color2['green'], color2['blue'])  # This is a grey color

                    #Create an IfcSurfaceStyleRendering that references the color
                    surface_style_rendering = ifc_file.createIfcSurfaceStyleShading()
                    surface_style_rendering.SurfaceColour = color 

                    # Create an IfcSurfaceStyle that holds the rendering
                    surface_style = ifc_file.createIfcSurfaceStyle(
                        color.Name,
                        'BOTH',
                        (surface_style_rendering,)
                    )
                    
                    presStyleAssign = ifc_file.createIfcPresentationStyleAssignment((surface_style,))
                    
                    #ifcopenshell..run("style.assign_representation_styles", ifc, shape_representation=representation_item, styles=[presStyleAssign])
                    # Create a new IfcStyledItem that associates the surface style to the wall's geometry

                    #Important!! we need to remove existing styles if set
                    for styledItem in representation_item.StyledByItem:
                        ifc_file.remove(styledItem)

                  
                    ifc_file.createIfcStyledItem(representation_item,(presStyleAssign,),color.Name)
                    
                    display(representation_item.StyledByItem)


ifc_file.write('c:\playground\path_to_modified_ifc_file5.ifc')

'IfcWallStandardCase'

(#454=IfcStyledItem(#195,(#453),$),)

'IfcWallStandardCase'

(#459=IfcStyledItem(#271,(#458),$),)

'IfcWallStandardCase'

(#464=IfcStyledItem(#303,(#463),$),)

'IfcWallStandardCase'

(#469=IfcStyledItem(#335,(#468),$),)